# Pivot Visualization

Interactive pivot detection visualization with window variations.

In [1]:
import plotly.graph_objects as go
import yfinance as yf

from indicators.pattern import find_pivots
from indicators.trend import calculate_ema, calculate_sma
from indicators.volatility import calculate_bbands

In [2]:
# Fetch data
ticker = yf.Ticker("AAPL")
df = ticker.history(start="2020-01-01", end="2024-01-01", auto_adjust=True)
df.columns = df.columns.str.lower()

print(f"Fetched {len(df)} bars for AAPL")

Fetched 1006 bars for AAPL


In [3]:
# Calculate indicators
sma_50 = calculate_sma(df, period=50)
sma_200 = calculate_sma(df, period=34)
ema_50 = calculate_ema(df, period=50)
ema_200 = calculate_ema(df, period=34)
bb_upper, bb_middle, bb_lower = calculate_bbands(df, period=13)

In [4]:
def plot_pivots(lb=8, rb=13, window_variations=None, title_suffix=""):
    """Plot price with pivots and indicators."""
    
    # Detect pivots
    pivot_high, pivot_low = find_pivots(
        df, lb=lb, rb=rb, return_boolean=True, 
        window_variations=window_variations, use_close=True
    )
    
    # Get pivot locations
    pivot_high_dates = df[pivot_high].index
    pivot_low_dates = df[pivot_low].index
    pivot_high_values = df.loc[pivot_high_dates, "close"]
    pivot_low_values = df.loc[pivot_low_dates, "close"]
    
    # Create figure
    fig = go.Figure()
    
    # Bollinger Bands
    fig.add_trace(go.Scatter(
        x=df.index, y=bb_upper, mode="lines", name="BB Upper",
        line=dict(color="gray", width=1, dash="dot"), opacity=0.5
    ))
    fig.add_trace(go.Scatter(
        x=df.index, y=bb_lower, mode="lines", name="BB Lower",
        line=dict(color="gray", width=1, dash="dot"), opacity=0.5,
        fill="tonexty", fillcolor="rgba(128, 128, 128, 0.1)"
    ))
    fig.add_trace(go.Scatter(
        x=df.index, y=bb_middle, mode="lines", name="BB Middle",
        line=dict(color="gray", width=1), opacity=0.5
    ))
    
    # SMAs
    fig.add_trace(go.Scatter(
        x=df.index, y=sma_50, mode="lines", name="SMA 50",
        line=dict(color="orange", width=1.2), opacity=0.7
    ))
    fig.add_trace(go.Scatter(
        x=df.index, y=sma_200, mode="lines", name="SMA 200",
        line=dict(color="purple", width=1.2), opacity=0.7
    ))
    
    # EMAs
    fig.add_trace(go.Scatter(
        x=df.index, y=ema_50, mode="lines", name="EMA 50",
        line=dict(color="green", width=1.2, dash="dash"), opacity=0.7
    ))
    fig.add_trace(go.Scatter(
        x=df.index, y=ema_200, mode="lines", name="EMA 200",
        line=dict(color="brown", width=1.2, dash="dash"), opacity=0.7
    ))
    
    # Price line
    fig.add_trace(go.Scatter(
        x=df.index, y=df["close"], mode="lines", name="Close",
        line=dict(color="black", width=1.5), opacity=0.8
    ))
    
    # Pivot highs
    fig.add_trace(go.Scatter(
        x=pivot_high_dates, y=pivot_high_values, mode="markers",
        name=f"Pivot High ({len(pivot_high_dates)})",
        marker=dict(color="red", size=6, line=dict(color="darkred", width=2)),
        text=[f"Close: ${v:.2f}" for v in pivot_high_values],
        hovertemplate="Date: %{x}<br>%{text}<extra></extra>"
    ))
    
    # Pivot lows
    fig.add_trace(go.Scatter(
        x=pivot_low_dates, y=pivot_low_values, mode="markers",
        name=f"Pivot Low ({len(pivot_low_dates)})",
        marker=dict(color="blue", size=6, line=dict(color="darkblue", width=2)),
        text=[f"Close: ${v:.2f}" for v in pivot_low_values],
        hovertemplate="Date: %{x}<br>%{text}<extra></extra>"
    ))
    
    # Layout
    var_str = f" variations={window_variations}" if window_variations else ""
    title = f"AAPL - Pivots (lb={lb}, rb={rb}{var_str}){title_suffix}"
    
    fig.update_layout(
        title=dict(text=title, font=dict(size=14)),
        xaxis_title="Date",
        yaxis_title="Price ($)",
        hovermode="x unified",
        template="plotly_white",
        height=800,
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    print(f"Detected: {len(pivot_high_dates)} highs, {len(pivot_low_dates)} lows")
    fig.show()

## View Pivot Labels

In [5]:
# Call find_pivots with window_variations
pivot_high, pivot_low = find_pivots(
    df, 
    lb=8, 
    rb=13, 
    return_boolean=True,
    window_variations=[-2, -1, 1, 2],
    use_close=True
)

# Create DataFrame to view labels
import pandas as pd
view_df = pd.DataFrame({
    'close': df['close'],
    'pivot_high': pivot_high,
    'pivot_low': pivot_low
})

# Show rows where pivots exist
view_df[view_df[['pivot_high', 'pivot_low']].any(axis=1)].head(20)

close  pivot_high  pivot_low
Date                                                       
2020-01-30 00:00:00-05:00  78.218925       False       True
2020-01-31 00:00:00-05:00  74.750786       False       True
2020-02-03 00:00:00-05:00  74.545486       False       True
2020-02-04 00:00:00-05:00  77.006523       False       True
2020-02-05 00:00:00-05:00  77.634445       False       True
2020-02-10 00:00:00-05:00  77.842903        True      False
2020-02-11 00:00:00-05:00  77.373238        True      False
2020-02-12 00:00:00-05:00  79.210701        True      False
2020-02-13 00:00:00-05:00  78.646629        True      False
2020-02-14 00:00:00-05:00  78.666008        True      False
2020-03-19 00:00:00-04:00  59.257919       False       True
2020-03-20 00:00:00-04:00  55.495907       False       True
2020-03-23 00:00:00-04:00  54.316933       False       True
2020-03-24 00:00:00-04:00  59.766304       False       True
2020-03-25 00:00:00-04:00  59.437073       False       True
2020-07-22 00:00:00-04:00  94.448402       False       True
2020-07-23 00:00:00-04:00  90.149460       False       True
2020-07-24 00:00:00-04:00  89.926109       False       True
2020-07-27 00:00:00-04:00  92.057404       False       True
2020-07-28 00:00:00-04:00  90.545120       False       True

## Base Configuration (lb=8, rb=13)

In [24]:
plot_pivots(lb=8, rb=13, window_variations=None)

Detected: 22 highs, 31 lows


## With ±1 Day Flexibility

In [6]:
plot_pivots(lb=8, rb=13, window_variations=[-1, 1])

Detected: 66 highs, 93 lows


## With ±2 Day Flexibility

In [7]:
plot_pivots(lb=8, rb=13, window_variations=[-2, -1, 1, 2])

Detected: 110 highs, 155 lows


## Symmetric Short Window (lb=5, rb=5)

In [8]:
plot_pivots(lb=5, rb=5, window_variations=None)

Detected: 55 highs, 55 lows


## Long Window (lb=13, rb=21)

In [9]:
plot_pivots(lb=13, rb=21, window_variations=None)

Detected: 16 highs, 21 lows


## Custom Configuration

Modify parameters below to test different settings.

In [11]:
# Customize these parameters
lb = 8
rb = 13
variations = [-2, -1, 1]

plot_pivots(lb=lb, rb=rb, window_variations=variations)

Detected: 88 highs, 124 lows
